In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm

In [ ]:
# Canada, Quebec City Dataframe (Main City)
df=pd.read_csv('../data/canada/listings.csv')
# Remove coloumn 'neighbourhood_group' and 'license'  because it's all NaN
df.drop(['neighbourhood_group','license'], axis=1, inplace=True)
# Show the shape
df.shape

In [ ]:
# Sample 10 Rows
df.sample(10)

In [ ]:
# Make a correlation heatmap
df.corr().style.background_gradient(cmap='coolwarm')

In [34]:
# Ireland Dataframe
dfIr=pd.read_csv('../data/ireland/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN
dfIr.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfIr.shape

(7566, 16)

In [35]:
# Sample 10 Rows
dfIr.sample(10)

,id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
5322,46270057,Locke Studio- Twin at Zanzibar Locke,371005518,Zanzibar Locke,Dublin City,53.347350,-6.265340,Entire home/apt,391,1,149,2022-08-28,7.00,7,357,140
435,6348110,Large double room in Castleknock,33067422,Ciara,Fingal,53.385040,-6.383420,Private room,40,3,24,2021-07-17,0.27,2,0,0
6333,612692390582103756,1 bedroom apartment to rent in dublin city center,148823725,Carla Brenda,Dublin City,53.329200,-6.277850,Entire home/apt,224,1,6,2022-07-03,1.49,1,5,6
5599,50963266,"Lovely modern 2 - bedroom apartment, in Dublin 1",376272590,Sovay,Dublin City,53.349903,-6.267653,Entire home/apt,130,15,7,2022-07-25,0.56,1,198,6
1818,16971923,Bright Comfortable King,26721551,Ann,Dublin City,53.311180,-6.286090,Private room,80,1,58,2022-08-27,0.87,2,272,10
6200,596835136364178143,Newly refurbished one bed self contained Apt,343981995,Helen,Dn Laoghaire-Rathdown,53.286230,-6.179500,Private room,118,2,12,2022-07-26,2.31,1,268,12
1465,14084572,Castleknock Apartment,84684668,Kieran,Fingal,53.373950,-6.404020,Entire home/apt,90,3,0,NaN,NaN,1,0,0
1849,17541932,Lovely flat in Dublin .,113256908,Barbra,Dublin City,53.365320,-6.255170,Entire home/apt,127,2,7,2018-10-08,0.10,1,0,0
2558,21089678,Superbly located! Bright and Airy Mews.,126495480,Daniel,Dublin City,53.335190,-6.245150,Entire home/apt,324,14,122,2022-08-21,2.02,6,334,6
641,7626133,Good Location-Spacious Double Room,40004472,Aoife,Dublin City,53.362550,-6.284130,Private room,61,1,9,2016-04-18,0.10,1,0,0


In [ ]:
# Make a correlation heatmap
dfIr.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Scotland Dataframe
dfSc=pd.read_csv('../data/scotland/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN
dfSc.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfSc.shape

In [ ]:
# Sample 10 Rows
dfSc.sample(10)

In [ ]:
# Make a correlation heatmap
dfSc.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Spain Dataframe
dfSp=pd.read_csv('../data/spain/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN
dfSp.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfSp.shape

In [ ]:
# Sample 10 Rows
dfSp.sample(10)

In [ ]:
# Make a correlation heatmap
dfSp.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Switzerland Dataframe
dfSw=pd.read_csv('../data/switzerland/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN 
dfSw.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfSw.shape

In [ ]:
# Sample 10 Rows
dfSw.sample(10)

In [ ]:
# Thailand Dataframe
dfTh=pd.read_csv('../data/thailand/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN 
dfTh.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfTh.shape

In [ ]:
# Sample 10 Rows
dfTh.sample(10)

In [ ]:
# Make a correlation heatmap
dfTh.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Turkey Dataframe
dfTu=pd.read_csv('../data/turkey/listings.csv')
# Remove coloumn 'neighbourhood_group' because it's all NaN 
dfTu.drop(['neighbourhood_group'], axis=1, inplace=True)
# Show the shape
dfTu.shape

In [ ]:
# Sample 10 Rows
dfTu.sample(10)

In [ ]:
# Make a correlation heatmap
dfTu.corr().style.background_gradient(cmap='coolwarm')